In [21]:
import librosa
import time

import soundfile as sf
import matplotlib.pyplot as plt
from IPython.display import Audio
import pandas as pd
import numpy as np 
import os
import soundfile as sf
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, auc

In [2]:
#get all features out of giver file 
def get_features(file_path, file_name):
    y, sr = librosa.load(file_path)
    iter_ = int(y.shape[0]/sr)
    features = []
    for i in range(iter_):
        y_seg = y[i*sr:(sr*i+sr)]
        chroma_stft = np.mean(librosa.feature.chroma_stft(y=y_seg, sr=sr))
        rms = np.mean(librosa.feature.rms(y=y_seg))
        spec_cent = np.mean(librosa.feature.spectral_centroid(y=y_seg, sr=sr))
        spec_bw = np.mean(librosa.feature.spectral_bandwidth(y=y_seg, sr=sr))
        rolloff = np.mean(librosa.feature.spectral_rolloff(y=y_seg, sr=sr))
        zcr = np.mean(librosa.feature.zero_crossing_rate(y_seg))
        mfcc = np.mean(librosa.feature.mfcc(y=y_seg, sr=sr, n_mfcc=20),axis=1)
        features.append([chroma_stft,rms,spec_cent,spec_bw,rolloff,zcr,*mfcc]) # starred expressions for mfcc to get 20 columns instead of 20 row
        # b = np.concatenate((b,a),axis=1) # ajout filename apres?
    
    return features
    

In [3]:
audio_dir='LA/'
# label = 'REAL'
folders = ['ASVspoof2019_LA_dev','ASVspoof2019_LA_eval','ASVspoof2019_LA_train']
sound_files_path = []
for folder in folders:
    files = os.listdir(os.path.join(audio_dir,folder,'flac'))
    for file in files:
        path = os.path.join(os.path.join(audio_dir,folder,'flac',file))
        if file.endswith('.flac'):
            sound_files_path.append(path)
len(sound_files_path) 
sound_files_path[0][28:41]

'LA_D_8848915.'

In [4]:
sound_files_path[0][28:40]

'LA_D_8848915'

In [13]:
#make dataframe with all the features
def make_features_df(audio_dir):
    feature_list = []
    for audio in audio_dir:
        filename = audio[28:40]
        print(f'{audio[28:40]} test')
        file_features=get_features(audio,filename)
        for segment_features in file_features:
                feature_list.append(segment_features + [filename])

    features_df = pd.DataFrame(feature_list,columns = data_set.columns)
    print(f'{filename} done')

    return features_df


In [15]:
len(sound_files_path)

122299

In [36]:
sound_files_path[0]

'LA/ASVspoof2019_LA_dev/flac/LA_D_8848915.flac'

In [35]:
# data_set = pd.read_csv('Data/DATASET-balanced.csv')
# start = time.time()
# asvs_features2 = make_features_df(sound_files_path[0:50])
# print('Time multiprocessing with joblib (many small tasks):', time.time() - start)

sound_files_path[122290:122299]

'LA/ASVspoof2019_LA_train/flac/LA_T_7164144.flac'

In [24]:
asvs_features2

,chroma_stft,rms,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,mfcc4,...,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20,LABEL
0,0.581642,0.003070,2199.394712,1951.288198,4489.428156,0.101729,-465.421295,125.339874,-69.007561,70.611092,...,-8.866219,7.612215,-12.527738,9.243306,-17.393263,6.024668,-14.883923,-1.570087,-11.686265,LA_D_8848915
1,0.325574,0.093229,2345.454133,1599.777611,3984.376665,0.152854,-236.496674,100.673485,-32.507786,49.088181,...,-10.290752,-2.104480,-12.366513,-0.074342,-20.750038,-0.219439,-13.774378,-12.227905,-15.365830,LA_D_8848915
2,0.315612,0.107650,1792.799111,1426.987016,3165.870250,0.124190,-233.423737,128.119858,-33.989712,45.122196,...,-4.690394,-8.381293,-15.633381,-2.263502,-20.402868,-5.021841,-14.528839,-10.487343,-13.769796,LA_D_8848915
3,0.539373,0.001211,1397.669671,1798.662060,3206.245006,0.058805,-601.982971,157.632462,-29.607780,64.629547,...,-18.353567,-2.086941,-21.357689,1.418289,-19.528841,2.797719,-17.868378,-0.484152,-12.906013,LA_D_4477471
4,0.309486,0.177945,1369.867761,1431.153215,2583.005593,0.078857,-275.831085,104.976555,-20.991016,46.207661,...,-12.673950,-6.094771,-14.902205,0.231349,-26.601341,-16.148132,-9.064847,-9.427711,-17.687447,LA_D_4477471
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153,0.392596,0.081438,1915.670360,1768.323318,4111.128928,0.069835,-390.356262,80.712227,-26.785952,65.946236,...,-7.936888,4.440732,-9.634704,-1.960031,2.161951,-5.973120,-12.247482,-2.869917,-8.838727,LA_D_4089906
154,0.368643,0.097091,1173.480353,1394.405909,2508.862860,0.050437,-334.047760,112.505302,-19.658857,80.507812,...,-15.980240,3.282106,-9.630474,-3.201415,4.315387,-6.114269,-15.845319,2.263654,-9.232529,LA_D_4089906
155,0.329175,0.128061,2011.860397,1662.137755,3804.525479,0.114779,-322.464874,83.027145,-15.504002,52.698841,...,-6.412160,-8.072599,-13.189572,-2.837007,-15.107281,-7.728273,-10.246255,-7.025140,-13.218804,LA_D_2536110
156,0.241618,0.127789,1368.052090,1300.213760,2558.046653,0.092651,-273.352173,141.598572,-27.962536,16.170324,...,-8.829967,-4.190058,-19.726473,-8.745008,-20.640242,-10.153530,-15.405114,-9.152931,-18.141642,LA_D_2536110


In [14]:
extra_features.to_csv('Data/Extra_features.csv',index=False)